# ToDo
* Referencing the original model
* Link einfügen (see below)
* Different approaches to create_instance => at least mention

# Motivation
The Pyomo-Documentation contains a [gallery](https://github.com/Pyomo/PyomoGallery/wiki), where the most popular introduction model to [GAMS](http://www.gams.com/) is rewritten using the Pyomo-package: [TRNSPRT](http://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/transport/transport.ipynb). 

However, in the pyomo gallery the model is created as a concrete model. This means, all sets and parameters need to be defined in the model file. For most practical applications it is desirable to create an abstract model that can be initialized with varying data, in order to use the model for more purposes. I'm going to show how to create the transport model as an abstract model, initialize it with the data of the standard example and solve the problem. I suggest you look through the example in the pyomo gallery before reading this post, because I won't repeat the explanation of some interesting details.

# The transport model
The transport model is an optimization of the transport costs of one good. Production capacities of the plants must not be exceeded, while the demand at the markets must be met. The transport costs are linear with respect to the freight and the distance between a plant and a market. For a description of the transport model using mathematical terms see the example in the [pyomo gallery](http://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/transport/transport.ipynb).

In the original description of the model, as well as in the replication in the pyomo gallery, simple characters are used as designators for sets, parameters and variables. While this might the mathematical convention, I'm going to deviate from that approach and use descriptive names.

# Implementation of the abstract model
The examples in the Pyomo gallery always import everything from `pyomo.environ`. In this blog, I'll only import the objects, that are actually needed. Firstly it is a better programming habit, since it reduces the risk of namespace collisions. But also, the scripts for model generation run faster. For the application to real world problems the importing time will be neglible compared to the model generation and solving time. But for simple problems it actually makes a difference. 

In contrast to the example in the pyomo gallery, the model must be an object of the class `ConcreteModel`.

In [1]:
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint, Objective, minimize
 
model = AbstractModel()

Sets, parameters, variables and constraints are defined using the same classes as for the concrete model. The only difference is, that the `initialize` parameter is left to be `None`. Even though I use descriptive names, I keep the `doc` parameter for model debugging purposes.

## Set definitions

In [2]:
model.plants = Set(doc="Canning plants")
model.markets = Set(doc="Markets")

## Parameter definitions
You can observe that using descriptive names will increase line length to a level where lines cannot be displayed in one line on standard environments. However, I value code readibility over fitting one line of commands in one actual line.

In [3]:
model.capacity_plant = Param(model.plants, doc='Capacity of a plant in cases')
model.demand_market = Param(model.markets, doc='Demand of a market in cases')
model.distance_in_thousand_miles = \
    Param(model.plants, model.markets, doc='Distance between plant and marekt in thousands of miles')
model.freight_costs_per_case_and_thousand_miles = \
    Param(initialize=90, doc='Freight in dollars per case per thousand miles')

Interestingly, you can still use the `initialize` parameter when adding parameters (the same holds true for sets/variables/constraints) to an abstract model. The function `freight_costs_per_case_in_thousands` is only invoked during the initialization of the concrete instance (see below). Therefore it can take the sets `model.plants` and `model.markets` as input, even though they are not yet initialized when the parameter `model.freight_costs_per_case_in_thousands` is defined.

In [4]:
def freight_costs_per_case_in_thousands(model, plant, market):
    return model.freight_costs_per_case_and_thousand_miles * model.distance_in_thousand_miles[plant,market] / 1000

model.freight_costs_per_case_in_thousands = Param(model.plants, 
                                                  model.markets, 
                                                  initialize=freight_costs_per_case_in_thousands, 
                                                  doc='Transport cost in thousands of dollar per case')

## Variable definitions

In [5]:
model.shipment_quantities_in_cases = Var(model.plants, model.markets, bounds=(0, None), doc='Shipment quantities in case')

## Constraints
Looking at list comprehensions, you can see how using descriptive names strongly enhances code readibility. When looking at lines like `sum(model.x[i,j] for j in model.j) <= model.a[i]`, I always have to look up the nomenclature. The code below, I can simply read and immediately understand what the constraint is doing. But I see how this is a matter of personal taste.

In [6]:
def supply_rule(model, plant):
  return sum(model.shipment_quantities_in_cases[plant,market] for market in model.markets) \
            <= model.capacity_plant[plant]

def demand_rule(model, market):
  return sum(model.shipment_quantities_in_cases[plant, market] for plant in model.plants) \
            >= model.demand_market[market] 

model.supply_constraint = Constraint(model.plants, rule=supply_rule, doc='Limit supply limit at each plant')
model.demand_constraint = Constraint(model.markets, rule=demand_rule, doc='Satisfy demand at each market')

## Objective

In [7]:
def objective_rule(model):
  return sum(model.freight_costs_per_case_in_thousands[plant,market] *
             model.shipment_quantities_in_cases[plant,market] 
                 for plant in model.plants for market in model.markets)

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

# Create a concrete instance of the abstract model
Up to now, the definition of the model didn't include any concrete data. To create an actual instance of the abstract model, you can simply call the function `create_instance()` on the abstract model's instance. It takes a file as input, that contains the data for solving the model in the [AMPL format](http://www.ampl.com/BOOK/CHAPTERS/12-data.pdf).

In [8]:
instance = model.create_instance(filename='trnsprt.dat')

For practical applications, you might want to create the abstract model in an own `.py` file and import the model to a script you use to initialize an instance, run that instance and do the post-processing. I'll create a post on how to do that soon.

# Running the instance
You can call the solve function of your `SolverFactory` instance on an abstract model's instance the same way as for a `ConcreteModel` instance.

In [9]:
from pyomo.opt import SolverFactory
opt = SolverFactory("glpk")
results = opt.solve(instance)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 153.675
  Upper bound: 153.675
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.019016027450561523
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

## Retrieving the output

In [10]:
def pyomo_postprocess(options=None, instance=None, results=None):
  instance.shipment_quantities_in_cases.display()

print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, instance, results)


Displaying Solution
------------------------------------------------------------
shipment_quantities_in_cases : Shipment quantities in case
    Size=6, Index=shipment_quantities_in_cases_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
     ('san-diego', 'chicago') :     0 :   0.0 :  None : False : False :  Reals
    ('san-diego', 'new-york') :     0 : 325.0 :  None : False : False :  Reals
      ('san-diego', 'topeka') :     0 : 275.0 :  None : False : False :  Reals
       ('seattle', 'chicago') :     0 : 300.0 :  None : False : False :  Reals
      ('seattle', 'new-york') :     0 :   0.0 :  None : False : False :  Reals
        ('seattle', 'topeka') :     0 :   0.0 :  None : False : False :  Reals
